In [1]:
import h5py
import pandas as pd
import numpy as np
import os

In [2]:
!pwd

/home/jovyan/workspace/Desktop/bsc_outputs/y2000_x4000_150_05_7particules


In [3]:
def populate_tables(data_folder_name='./data', 
                   tracers='tip'):
    #########################################
    """
    This function create the txt files with the strain rate data
    
    Arguments:
    - data_folder_name (str): folder where the raw h5 files are stored. 
        Default is 'outputs-fixed-values'.
    - tracers (str): name of tracers. Default is 'lag'.
    
    Returns:
    - Folder called txt_outputs with all the files.
    """
    #########################################
    
    # Check if the data folder exists
    if not os.path.exists(data_folder_name):
        raise FileNotFoundError(f"Folder '{data_folder_name}' not found.")
    
    # count number of times, checking the number that files that starts with prefix
    prefix = f'{tracers}_velocitygradient'
    files = os.listdir(data_folder_name)
    num_files = sum([1 for file in files if file.startswith(prefix)])
    print(num_files)
    # loop by number of files
    time = 0
    while time < num_files:
        print(f'time: {time}')
        ## COORDINATES ##
        # open Coordinates H5 
        h5_file_coord =  f'./{data_folder_name}/{tracers}-{time}.h5'
        if not os.path.exists(h5_file_coord):
                print(f"Warning: H5 file not found at '{h5_file_coord}'. Skipping this time step.")
                time += 1
                continue
        
        with h5py.File(h5_file_coord, 'r') as file:
            # create a variable name to store coord
            dataset_name = f'np_array_{time}'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                globals()[dataset_name] = file[key][:] 
                
        # create a table for each time and populate them with numpy array 
        # first create the name of the table
        df_name_by_t = f'df_t_{time}'
        # create the table
        globals()[df_name_by_t] = pd.DataFrame(globals()[dataset_name], columns=['x', 'y', 'z'])
        #add index in first column
        globals()[df_name_by_t].insert(0, 'index', globals()[df_name_by_t].index)
        #print(df_name_by_t)

        ## Velocity Gradient TENSOR ##
        # open the Strain Rate Tensor H5 files 
        h5_file_path =  f'./{data_folder_name}/{prefix}-{time}.h5'
        if not os.path.exists(h5_file_path):
                print(f"Warning: H5 Strain Rate Tensor file not found at '{h5_file_path}'. Skipping this time step.")
                time += 1
                continue
        
        with h5py.File(h5_file_path, 'r') as file:
            # create a variable name for storing strain rate tensor array
            dataset_name = f'np_array_{time}'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                globals()[dataset_name] = file[key][:] 
            
        # [a00,a01,a02,a10,a11,a12,a20,a21,a22]
        #globals()[dataset_name][0]
        globals()[df_name_by_t]['VG00'] = globals()[dataset_name][:, 0]
        globals()[df_name_by_t]['VG01'] = globals()[dataset_name][:, 1]
        globals()[df_name_by_t]['VG02'] = globals()[dataset_name][:, 2]
        globals()[df_name_by_t]['VG10'] = globals()[dataset_name][:, 3]
        globals()[df_name_by_t]['VG11'] = globals()[dataset_name][:, 4]
        globals()[df_name_by_t]['VG12'] = globals()[dataset_name][:, 5]
        globals()[df_name_by_t]['VG20'] = globals()[dataset_name][:, 6]
        globals()[df_name_by_t]['VG21'] = globals()[dataset_name][:, 7]
        globals()[df_name_by_t]['VG22'] = globals()[dataset_name][:, 8]
          
        ## save the df as txt
        output_folder = 'txt_outputs_test'
        output_file = os.path.join(output_folder, f'results{df_name_by_t}.txt')
            
        # check if the folder exists; if not, create it
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
            print('out folder: ', output_folder)
            
        # save to tab-separated values (.txt) file
        globals()[df_name_by_t].to_csv(output_file, sep='\t', index=False)

        #########
            
            
        print(f'hi {df_name_by_t}') 
        print(globals()[df_name_by_t].tail())
        time += 1
        
            
        

In [4]:
populate_tables()

301
time: 0
out folder:  txt_outputs_test
hi df_t_0
   index            x            y      z  VG00  VG01  VG02  VG10  VG11  VG12  \
0      0  1600.000000   455.555556 -400.0   0.0   0.0   0.0   0.0   0.0   0.0   
1      1  1733.333333   922.222222 -400.0   0.0   0.0   0.0   0.0   0.0   0.0   
2      2  1600.000000  1544.444444 -400.0   0.0   0.0   0.0   0.0   0.0   0.0   

   VG20  VG21  VG22  
0   0.0   0.0   0.0  
1   0.0   0.0   0.0  
2   0.0   0.0   0.0  
time: 1
hi df_t_1
   index            x            y           z          VG00          VG01  \
0      0  1600.127265   454.983226 -401.035265  7.272704e-17  2.572040e-17   
1      1  1733.582659   921.933299 -400.594282 -7.387846e-17  1.406417e-17   
2      2  1600.522062  1544.464877 -400.918195  9.572418e-20 -7.543371e-18   

           VG02          VG10          VG11          VG12          VG20  \
0  8.171345e-16 -1.658829e-17  1.972822e-17  1.252886e-16  3.680964e-16   
1  4.649360e-16  3.551036e-17 -9.268747e-18  9.973965e

hi df_t_38
   index            x            y           z          VG00          VG01  \
0      0  1606.736812   446.115860 -425.002968 -1.046800e-16  3.341873e-17   
1      1  1740.797424   921.005876 -417.555825 -2.321611e-17 -4.691933e-18   
2      2  1608.598382  1551.307835 -425.192508 -1.081363e-16 -4.481313e-17   

           VG02          VG10          VG11          VG12          VG20  \
0  4.284845e-16 -5.082647e-17  7.275095e-17  9.871604e-17  1.420471e-16   
1  3.049301e-16 -1.413957e-17  7.112599e-18  7.402966e-18  1.633838e-16   
2  4.554490e-16  5.045518e-17  7.794164e-17 -1.335349e-16  1.611014e-16   

           VG21          VG22  
0 -8.577613e-17  4.421780e-17  
1  5.146920e-18  2.443913e-17  
2  7.539469e-17  4.128031e-17  
time: 39
hi df_t_39
   index            x            y           z          VG00          VG01  \
0      0  1606.910383   445.854137 -425.815633 -1.031851e-16  8.741588e-17   
1      1  1740.993465   920.944442 -418.143722 -3.168203e-17  8.447833e

hi df_t_77
   index            x            y           z          VG00          VG01  \
0      0  1610.485604   433.556936 -455.787125 -6.450837e-18  5.944256e-17   
1      1  1749.746182   919.842198 -442.562128  6.487593e-17  1.237138e-17   
2      2  1612.257013  1563.026156 -456.122172  5.011587e-17 -2.418565e-17   

           VG02          VG10          VG11          VG12          VG20  \
0  7.124170e-16 -9.672235e-17  8.062809e-17  1.581153e-16 -1.541284e-16   
1  2.827536e-16 -3.587910e-18  1.096494e-17  4.680937e-17  4.518437e-16   
2  6.920268e-16  8.171337e-17  4.586438e-17 -2.558160e-17 -3.379807e-16   

           VG21          VG22  
0 -1.137726e-16 -1.321099e-16  
1 -2.319204e-17  9.731655e-17  
2  1.060283e-16 -1.814817e-16  
time: 78
hi df_t_78
   index            x            y           z          VG00          VG01  \
0      0  1610.484268   433.106392 -456.632766 -8.102190e-17  2.751068e-16   
1      1  1750.004316   919.804578 -443.274884  4.663232e-17  1.517933e

2  3.914216e-17  4.853300e-17  
time: 118
hi df_t_118
   index            x            y           z          VG00          VG01  \
0      0  1610.264379   419.393715 -483.765136 -2.399055e-17  6.276505e-17   
1      1  1758.224528   918.348731 -466.310487 -6.172461e-17 -3.155086e-18   
2      2  1612.082364  1576.342808 -483.961700 -9.095429e-17 -1.398479e-16   

           VG02          VG10          VG11          VG12          VG20  \
0  6.724824e-16 -7.641005e-17  7.143681e-17  6.008120e-17  1.523542e-16   
1  3.743956e-16  5.015624e-18  3.380717e-17 -6.640895e-17  7.929222e-17   
2  1.212597e-15  2.326023e-16  1.129718e-16 -6.929093e-16  5.750941e-17   

           VG21          VG22  
0 -7.532334e-17  4.412830e-18  
1 -2.168284e-18 -2.036323e-17  
2  7.219211e-18  5.957962e-17  
time: 119
hi df_t_119
   index            x            y           z          VG00          VG01  \
0      0  1610.186443   419.126355 -484.342321 -3.535867e-17  7.751077e-17   
1      1  1758.322383   91

hi df_t_158
   index            x            y           z          VG00          VG01  \
0      0  1596.892211   407.599608 -507.098384  8.054330e-17  6.817365e-17   
1      1  1600.229797  1586.435080 -508.866804  1.097802e-16 -2.872582e-17   
2      2  1758.923615   916.269516 -489.642922  4.764981e-17  8.984703e-18   

           VG02          VG10          VG11          VG12          VG20  \
0  6.529632e-16 -1.679824e-16  5.303218e-17  1.115859e-16  2.764802e-17   
1  5.837022e-16  1.176452e-16  5.051574e-17 -1.026521e-16  1.676238e-17   
2  4.201537e-16 -1.522510e-17  1.559723e-17  4.569095e-17  2.076426e-17   

           VG21          VG22  
0 -6.841727e-17 -1.785158e-16  
1  5.924479e-17 -1.683085e-16  
2 -8.187894e-18 -4.974659e-17  
time: 159
hi df_t_159
   index            x            y           z          VG00          VG01  \
0      0  1596.346786   407.296669 -507.727629  1.057112e-16  5.407300e-17   
1      1  1599.734875  1586.621951 -509.428237  1.128819e-16 -2.1301

2  7.146635e-17 -1.897319e-16  
time: 199
hi df_t_199
   index            x            y           z          VG00          VG01  \
0      0  1578.651390   398.210845 -532.221107 -5.912142e-17  6.444577e-16   
1      1  1756.885318   914.762525 -528.710605 -1.900199e-17  1.264861e-17   
2      2  1589.717001  1595.999051 -534.783082  1.407644e-17 -1.834408e-16   

           VG02          VG10          VG11          VG12          VG20  \
0  1.694756e-15 -1.804783e-16  1.623017e-16  4.225091e-16 -8.515760e-17   
1  5.057531e-16 -7.106762e-18  1.549679e-17 -1.002216e-17  7.714736e-17   
2  2.381928e-15  1.500649e-16  2.586883e-16 -6.264878e-16 -1.088520e-16   

           VG21          VG22  
0 -2.087249e-16 -1.927573e-16  
1 -1.802152e-18 -7.554309e-17  
2  1.355156e-16 -1.979936e-16  
time: 200
hi df_t_200
   index            x            y           z          VG00          VG01  \
0      0  1578.475273   398.244057 -532.761062 -2.931859e-17  6.524765e-16   
1      1  1756.806333   91

hi df_t_238
   index            x            y           z          VG00          VG01  \
0      0  1552.136297   395.286318 -550.805160  2.017791e-16  5.454636e-17   
1      1  1744.519506   913.673863 -561.955127  2.479108e-16  1.720200e-17   
2      2  1561.381966  1601.677485 -553.756497  2.166755e-16 -1.005179e-16   

           VG02          VG10          VG11          VG12          VG20  \
0  2.049932e-15 -1.338085e-16  1.794422e-17  4.849825e-16 -1.708405e-16   
1  1.002790e-15 -1.905383e-17  5.065032e-17  6.648132e-17 -2.435111e-17   
2  1.989536e-15  5.964330e-17  4.243272e-17 -6.836751e-16 -1.740950e-16   

           VG21          VG22  
0  5.165408e-17 -1.894484e-16  
1  1.709440e-18 -2.881361e-17  
2 -7.896535e-18 -2.054469e-16  
time: 239
hi df_t_239
   index            x            y           z          VG00          VG01  \
0      0  1551.309256   395.484342 -551.279237  2.813023e-16  3.337514e-17   
1      1  1743.775497   913.810801 -562.814607  3.096516e-16  5.5336

hi df_t_278
   index            x            y           z          VG00          VG01  \
0      0  1506.940311   394.779395 -562.095145  7.326317e-16  1.193305e-15   
1      1  1710.433921   912.325294 -587.838957  3.866756e-16 -5.361321e-17   
2      2  1516.325053  1601.292235 -566.595994  8.267179e-16  1.037197e-16   

           VG02          VG10          VG11          VG12          VG20  \
0  5.717831e-16 -6.867246e-17 -2.106617e-16 -1.584831e-16 -3.874106e-16   
1  1.096240e-15  1.692311e-17 -2.640254e-17 -5.736207e-17 -7.958043e-17   
2  3.705647e-15 -3.390465e-16 -5.874263e-16 -7.423404e-16 -6.944408e-16   

           VG21          VG22  
0  8.237984e-17 -4.710573e-16  
1 -3.213611e-17 -3.751086e-16  
2  3.027098e-16 -1.992057e-16  
time: 279
hi df_t_279
   index            x            y           z          VG00          VG01  \
0      0  1506.961442   395.272895 -562.113822  6.631706e-16  1.115009e-15   
1      1  1708.894730   912.351591 -588.414674  3.784316e-16 -1.8739

In [8]:
h5_file_coord =  f'./outputs_3d_no_t_copy/lag_velocitygradient-1.h5'

with h5py.File(h5_file_coord, 'r') as file:
            # create a variable name for storing strain rate tensor array
            dataset_name = f'np_array_6'

            for key in file.keys():
                # create a string with the name of the var, and assign numpy value from h5
                 dataset_name = file[key][:] 
                    

dataset_name[0]

HDF5-DIAG: Error detected in HDF5 (1.12.1) thread 0:
  #000: H5F.c line 620 in H5Fopen(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #001: H5VLcallback.c line 3501 in H5VL_file_open(): failed to iterate over available VOL connector plugins
    major: Virtual Object Layer
    minor: Iteration failed
  #002: H5PLpath.c line 578 in H5PL__path_table_iterate(): can't iterate over plugins in plugin path '(null)'
    major: Plugin for dynamically loaded library
    minor: Iteration failed
  #003: H5PLpath.c line 620 in H5PL__path_table_iterate_process_path(): can't open directory: /usr/local/hdf5/lib/plugin
    major: Plugin for dynamically loaded library
    minor: Can't open directory or file
  #004: H5VLcallback.c line 3351 in H5VL__file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #005: H5VLnative_file.c line 97 in H5VL__native_file_open(): unable to open file
    major: File accessibility
    minor: Unable to o

RuntimeError: Unspecified error in H5Fopen (return value <0)